## Example to serve our model and do A/B testing with Canary rollouts

Install KF serving, if not available

In [ ]:
!pip install kfserving==0.3.0.1 --user

In [ ]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements



Define namespace for deployment

In [ ]:
namespace = utils.get_default_target_namespace()

Define the endpoint specification

In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            min_replicas=1,
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='gs://smart_movement//model_v1',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'0.5Gi'},
                                  limits={'cpu':'100m', 'memory':'0.5Gi'}))))

isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='flowers-sample', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

Create the inference service from the specification

In [ ]:
kf_serving = KFServingClient()
kf_serving.create(isvc)